<a href="https://colab.research.google.com/github/liachicue/HOG_ML/blob/main/HOG_ML_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación de un Clasificador SVM con PCA en Histogramas de Gradientes Orientados

**Aprendizaje de Máquina**

**Universidad Tecnológica de Pereira**


Lia Natalia Chicue Garcia 

*2021-II*

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import  PCA
from ipywidgets import  interact
from sklearn.svm import SVC 
from sklearn.model_selection import  train_test_split
from sklearn.metrics import  accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import scale

import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving HOG1.xlsx to HOG1.xlsx


In [ ]:
dfCaracteristicas= pd.read_excel('HOG1.xlsx')
dfCaracteristicas

,Etiquetas,Carcaterísticas 1,Carcaterísticas 2,Carcaterísticas 3,Carcaterísticas 4,Carcaterísticas 5,Carcaterísticas 6,Carcaterísticas 7,Carcaterísticas 8,Carcaterísticas 9,Carcaterísticas 10,Carcaterísticas 11,Carcaterísticas 12,Carcaterísticas 13,Carcaterísticas 14,Carcaterísticas 15,Carcaterísticas 16,Carcaterísticas 17,Carcaterísticas 18,Carcaterísticas 19,Carcaterísticas 20,Carcaterísticas 21,Carcaterísticas 22,Carcaterísticas 23,Carcaterísticas 24,Carcaterísticas 25,Carcaterísticas 26,Carcaterísticas 27,Carcaterísticas 28,Carcaterísticas 29,Carcaterísticas 30,Carcaterísticas 31,Carcaterísticas 32,Carcaterísticas 33,Carcaterísticas 34,Carcaterísticas 35,Carcaterísticas 36,Carcaterísticas 37,Carcaterísticas 38,Carcaterísticas 39,...,Carcaterísticas 8061,Carcaterísticas 8062,Carcaterísticas 8063,Carcaterísticas 8064,Carcaterísticas 8065,Carcaterísticas 8066,Carcaterísticas 8067,Carcaterísticas 8068,Carcaterísticas 8069,Carcaterísticas 8070,Carcaterísticas 8071,Carcaterísticas 8072,Carcaterísticas 8073,Carcaterísticas 8074,Carcaterísticas 8075,Carcaterísticas 8076,Carcaterísticas 8077,Carcaterísticas 8078,Carcaterísticas 8079,Carcaterísticas 8080,Carcaterísticas 8081,Carcaterísticas 8082,Carcaterísticas 8083,Carcaterísticas 8084,Carcaterísticas 8085,Carcaterísticas 8086,Carcaterísticas 8087,Carcaterísticas 8088,Carcaterísticas 8089,Carcaterísticas 8090,Carcaterísticas 8091,Carcaterísticas 8092,Carcaterísticas 8093,Carcaterísticas 8094,Carcaterísticas 8095,Carcaterísticas 8096,Carcaterísticas 8097,Carcaterísticas 8098,Carcaterísticas 8099,Carcaterísticas 8100
0,Abuelo,0.145068,0.158248,0.100790,0.200074,0.036291,0.072511,0.131387,0.126176,-0.002905,0.183430,0.023815,0.160107,0.216108,0.110930,0.097971,0.213105,-4.746912e-02,8.361223e-02,1.396469e-01,2.031732e-02,0.497431,0.115566,0.302340,0.186030,0.020513,0.371307,0.168252,0.054149,0.088051,0.113869,-0.004325,0.115875,0.181511,0.062840,0.138816,0.119003,0.109991,0.218338,0.113399,...,0.038037,0.040859,0.003076,0.128472,0.001083,0.012801,-0.000444,-0.001395,-0.002676,0.034400,0.017125,-0.010488,0.011018,-0.000748,-0.088492,0.063747,-0.003852,-0.003558,0.309523,0.110396,-0.063941,0.034261,2.652976e-01,-0.002116,-0.101698,-0.062526,0.083263,0.046218,0.798434,0.051926,0.003835,0.017465,0.126256,0.324046,0.005354,0.000419,0.003153,0.131690,0.005395,0.018898
1,Abuelo,0.000520,0.008116,0.117544,0.631401,-0.007397,0.011709,-0.013694,0.045847,0.014660,0.017221,0.024517,0.087705,-0.028613,0.011510,0.010549,0.084493,-2.498088e-02,5.382424e-03,-1.392643e-02,2.765293e-02,0.525333,0.367627,0.350446,-0.017839,0.040082,0.117092,0.084467,0.069304,-0.003517,0.003249,0.029795,-0.050688,0.023348,0.021404,0.008599,0.016635,0.127212,0.683333,0.006090,...,0.015717,-0.001270,0.004462,-0.015857,0.000000,0.097995,0.010877,-0.005902,0.000000,0.050962,-0.003581,0.018047,-0.002816,0.075822,0.016850,0.035384,0.003157,-0.072275,0.019058,-0.005656,-0.233100,0.173100,-7.310919e-03,-0.100164,0.561062,0.134809,-0.015401,0.109201,0.466439,0.093520,0.196669,0.500562,0.018319,0.047141,0.058081,0.105863,0.003789,-0.013466,0.000739,0.044471
2,Abuelo,0.247088,-0.024690,0.204334,-0.080550,0.104009,0.101327,0.053787,0.144329,0.012001,-0.017416,-0.011251,0.439679,0.027006,0.186102,0.130934,0.301618,1.146631e-02,5.659269e-02,-2.258399e-02,3.244082e-01,0.000952,0.419108,0.090328,0.109830,0.015436,0.116937,-0.003515,0.097391,0.154661,0.186795,-0.071215,0.110629,0.026182,0.061554,0.265069,0.169355,0.162318,-0.063987,0.288768,...,0.287670,0.141455,0.051845,0.010705,0.008179,0.055281,0.110831,0.061315,0.086034,0.220864,-0.067191,-0.015828,0.136101,0.240489,0.286483,0.055639,0.688299,0.130686,0.232661,0.138230,0.146400,0.043052,1.322608e-02,0.134917,0.051763,0.026652,-0.059881,0.099581,0.056418,0.022320,0.243061,0.009589,0.045114,0.017999,0.196550,0.062870,0.061213,0.012640,0.153316,0.022049
3,Abuelo,0.020921,-0.017220,0.037705,0.089019,-0.025870,0.018287,0.004603,0.006051,0.038234,0.018090,0.145809,0.035883,0.099371,0.03061

In [ ]:
X = dfCaracteristicas.drop(columns=["Etiquetas"])
X.head()

,Carcaterísticas 1,Carcaterísticas 2,Carcaterísticas 3,Carcaterísticas 4,Carcaterísticas 5,Carcaterísticas 6,Carcaterísticas 7,Carcaterísticas 8,Carcaterísticas 9,Carcaterísticas 10,Carcaterísticas 11,Carcaterísticas 12,Carcaterísticas 13,Carcaterísticas 14,Carcaterísticas 15,Carcaterísticas 16,Carcaterísticas 17,Carcaterísticas 18,Carcaterísticas 19,Carcaterísticas 20,Carcaterísticas 21,Carcaterísticas 22,Carcaterísticas 23,Carcaterísticas 24,Carcaterísticas 25,Carcaterísticas 26,Carcaterísticas 27,Carcaterísticas 28,Carcaterísticas 29,Carcaterísticas 30,Carcaterísticas 31,Carcaterísticas 32,Carcaterísticas 33,Carcaterísticas 34,Carcaterísticas 35,Carcaterísticas 36,Carcaterísticas 37,Carcaterísticas 38,Carcaterísticas 39,Carcaterísticas 40,...,Carcaterísticas 8061,Carcaterísticas 8062,Carcaterísticas 8063,Carcaterísticas 8064,Carcaterísticas 8065,Carcaterísticas 8066,Carcaterísticas 8067,Carcaterísticas 8068,Carcaterísticas 8069,Carcaterísticas 8070,Carcaterísticas 8071,Carcaterísticas 8072,Carcaterísticas 8073,Carcaterísticas 8074,Carcaterísticas 8075,Carcaterísticas 8076,Carcaterísticas 8077,Carcaterísticas 8078,Carcaterísticas 8079,Carcaterísticas 8080,Carcaterísticas 8081,Carcaterísticas 8082,Carcaterísticas 8083,Carcaterísticas 8084,Carcaterísticas 8085,Carcaterísticas 8086,Carcaterísticas 8087,Carcaterísticas 8088,Carcaterísticas 8089,Carcaterísticas 8090,Carcaterísticas 8091,Carcaterísticas 8092,Carcaterísticas 8093,Carcaterísticas 8094,Carcaterísticas 8095,Carcaterísticas 8096,Carcaterísticas 8097,Carcaterísticas 8098,Carcaterísticas 8099,Carcaterísticas 8100
0,0.145068,0.158248,0.100790,0.200074,0.036291,0.072511,0.131387,0.126176,-0.002905,0.183430,0.023815,0.160107,0.216108,0.110930,0.097971,0.213105,-4.746912e-02,0.083612,0.139647,0.020317,0.497431,0.115566,0.302340,0.186030,0.020513,0.371307,0.168252,0.054149,0.088051,0.113869,-0.004325,0.115875,0.181511,0.062840,0.138816,0.119003,0.109991,0.218338,0.113399,0.163682,...,0.038037,0.040859,0.003076,0.128472,0.001083,0.012801,-0.000444,-0.001395,-0.002676,0.034400,0.017125,-0.010488,0.011018,-0.000748,-0.088492,0.063747,-0.003852,-0.003558,0.309523,0.110396,-0.063941,0.034261,0.265298,-0.002116,-0.101698,-0.062526,0.083263,0.046218,0.798434,0.051926,0.003835,0.017465,0.126256,0.324046,0.005354,0.000419,0.003153,0.131690,0.005395,0.018898
1,0.000520,0.008116,0.117544,0.631401,-0.007397,0.011709,-0.013694,0.045847,0.014660,0.017221,0.024517,0.087705,-0.028613,0.011510,0.010549,0.084493,-2.498088e-02,0.005382,-0.013926,0.027653,0.525333,0.367627,0.350446,-0.017839,0.040082,0.117092,0.084467,0.069304,-0.003517,0.003249,0.029795,-0.050688,0.023348,0.021404,0.008599,0.016635,0.127212,0.683333,0.006090,0.547389,...,0.015717,-0.001270,0.004462,-0.015857,0.000000,0.097995,0.010877,-0.005902,0.000000,0.050962,-0.003581,0.018047,-0.002816,0.075822,0.016850,0.035384,0.003157,-0.072275,0.019058,-0.005656,-0.233100,0.173100,-0.007311,-0.100164,0.561062,0.134809,-0.015401,0.109201,0.466439,0.093520,0.196669,0.500562,0.018319,0.047141,0.058081,0.105863,0.003789,-0.013466,0.000739,0.044471
2,0.247088,-0.024690,0.204334,-0.080550,0.104009,0.101327,0.053787,0.144329,0.012001,-0.017416,-0.011251,0.439679,0.027006,0.186102,0.130934,0.301618,1.146631e-02,0.056593,-0.022584,0.324408,0.000952,0.419108,0.090328,0.109830,0.015436,0.116937,-0.003515,0.097391,0.154661,0.186795,-0.071215,0.110629,0.026182,0.061554,0.265069,0.169355,0.162318,-0.063987,0.288768,0.227231,...,0.287670,0.141455,0.051845,0.010705,0.008179,0.055281,0.110831,0.061315,0.086034,0.220864,-0.067191,-0.015828,0.136101,0.240489,0.286483,0.055639,0.688299,0.130686,0.232661,0.138230,0.146400,0.043052,0.013226,0.134917,0.051763,0.026652,-0.059881,0.099581,0.056418,0.022320,0.243061,0.009589,0.045114,0.017999,0.196550,0.062870,0.061213,0.012640,0.153316,0.022049
3,0.020921,-0.017220,0.037705,0.089019,-0.025870,0.018287,0.004603,0.006051,0.038234,0.018090,0.145809,0.035883,0.099371,0.030617,0.155962,0.017379,6.865253e-02,0.03716

In [ ]:
y = dfCaracteristicas["Etiquetas"].astype('category').cat.codes 
y.unique()

array([0, 2, 1], dtype=int8)

## Implementación y Evaluación del Modelo SVC sin PCA 

Se utilizará el toolkit de sklearn llamado [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), el cual es capaz de clasificar bases de datos multi-clase apoyado en métodos de aprendizaje supervisado como lo son las máquinas de soporte vectorial.


Se evaluará la precisión de la clasificación (`accuracy_score`) sujeto a los valores;

* Tamaño de prueba (`test_size`), el cual determina la proporción del conjunto de datos que se incluirá en la división de prueba.
* Los hiperparámetros;
 * Kernel polinómico (`kernel = poly`), El kernel específica el tipo de kernel que se utilizará en el algoritmo. En nuestro caso utilizaremos un kernel polinómico.
 * Grado de los polinomios de la función Kernel (`Deg`). 
 * Parámetro de Regularización (`C`)


In [ ]:
#Dividimos los valores de Prueba y Entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 ,random_state = 123)


#Implementamos el Modelo SVC
@interact(kernel = ['linear','rbf','poly','sigmoid'],C = [0.01,1,10,100,1e6],Deg =[2,3,6])
def SVMclass(kernel,C,Deg):
  model = SVC(kernel=kernel,C=C,degree=Deg)
  model.fit(X_train,y_train)
  yEst = model.predict(X_test)
  Acc_test = accuracy_score(y_test, yEst)

  print("\n Con el Kernel:", kernel)
  print("\n Con Regulariacion:", C)
  print("\n Con Grados para el kernel polinómico:", Deg)
  print('\n Precision de la Clasificación: ',Acc_test*100)

  # Evaluamos el desempeño del clasificador 
  target_names = dfCaracteristicas['Etiquetas'].unique()
  print("\n Evaluamos el comportamiento de la prediccion \n", classification_report(y_test, yEst, target_names=target_names))
  cMat_test = confusion_matrix(y_test, yEst)

  ax2 = ConfusionMatrixDisplay(cMat_test, display_labels = target_names)
  ax2.plot()
  plt.show()

interactive(children=(Dropdown(description='kernel', options=('linear', 'rbf', 'poly', 'sigmoid'), value='line…

## Implementación y Evaluación del Modelo SVC con PCA

### PCA

Creamos una función Interactiva que permite almacenar los valores de la base de datos con PCA para n = 2, n = 5 y n = 11. 

In [ ]:
@interact(M =[2,5,11])
def ExpPCA(M=11):
  X = dfCaracteristicas.drop(columns=["Etiquetas"])
  modelPCA = PCA(n_components=M)
  modelPCA.fit(X)
  X_nuevo = modelPCA.transform(X)#Reescribo la variable
  print(X_nuevo)
  HOG_PCA_M = pd.DataFrame(X_nuevo)
  HOG_PCA_M["Etiquetas"] = dfCaracteristicas["Etiquetas"]
  HOG_PCA_M.to_excel(f"HOG_PCA_{M}.xlsx", index=False)
  HOG_PCA_M.head()

interactive(children=(Dropdown(description='M', index=2, options=(2, 5, 11), value=11), Output()), _dom_classe…

### Modelo SVC

In [ ]:
#Seleccionamos la de Base de datos a Uilizar
dfHOG_PCA_M = pd.read_excel('/content/HOG_PCA_5.xlsx')

#Separamos las caracteristicas de los targets
X_a = dfHOG_PCA_M.drop(columns=["Etiquetas"])
Y_a = dfHOG_PCA_M["Etiquetas"].astype('category').cat.codes

#Dividimos los valores de Prueba y Entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X_a, Y_a, test_size = 0.3 ,random_state = 123)


#Implementamos el Modelo SVC
@interact(kernel = ['linear','rbf','poly','sigmoid'],C = [0.01,1,10,100,1e6],Deg =[2,3,6])
def SVMclass(kernel,C,Deg):
  model = SVC(kernel=kernel,C=C,degree=Deg)
  model.fit(X_train,y_train)
  yEst = model.predict(X_test)
  Acc_test = accuracy_score(y_test, yEst)
  
  print("\n Con el Kernel:", kernel)
  print("\n Con Regulariacion:", C)
  print("\n Con Grados para el kernel polinómico:", Deg)

  # Evaluamos el desempeño del clasificador 
  target_names = dfCaracteristicas['Etiquetas'].unique()
  print('\n Precision de la Clasificación: ',Acc_test*100)
  print("\n Evaluamos el comportamiento de la prediccion \n", classification_report(y_test, yEst, target_names=target_names))
  cMat_test = confusion_matrix(y_test, yEst)

  ax2 = ConfusionMatrixDisplay(cMat_test, display_labels = target_names)
  ax2.plot()
  plt.show()

interactive(children=(Dropdown(description='kernel', options=('linear', 'rbf', 'poly', 'sigmoid'), value='line…